In [175]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, RidgeClassifier #,Ridge,Lasso,ElasticNet
from sklearn.metrics import accuracy_score,recall_score,precision_score,roc_auc_score,f1_score
from sklearn.model_selection import train_test_split#,cross_val_score,KFold,RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import NearMiss
from sklearn.datasets import make_classification
import numpy as np
from collections import Counter

In [176]:
cd

C:\Users\37789


In [177]:
df = pd.read_parquet('fraud_detection.parquet')

In [178]:
df.drop(columns = ['SK_ID_CURR','AMT_GOODS_PRICE','CODE_GENDER'],inplace = True)

In [179]:

object_cols = df.select_dtypes(include=['object']).columns

In [180]:
df[object_cols] = df[object_cols].apply(lambda x: x.str.upper())

In [181]:
if (df['AMT_ANNUITY'].dtypes == 'int64') | (df['AMT_ANNUITY'].dtypes == 'float64'):
    print('x')

x


In [182]:
def drop_high_nulls_col(df):
    # Grouped median calculation for filling missing values
    
    for cols in df.columns:
        if (df[cols].notnull().sum() / df.shape[0]) < 0.4:
            df.drop(columns=[cols], inplace=True)
            
    return df

In [183]:
df = drop_high_nulls_col(df)

In [184]:
X_col = df.drop(columns = 'TARGET').columns
y_col = ['TARGET']

In [185]:
group_cols = ['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']

In [186]:
X_train,X_test,y_train,y_test = train_test_split(df[X_col],df[y_col],stratify=df[group_cols],train_size=0.8,test_size = 0.2,random_state=37)

In [187]:
X_train.to_parquet('X_train.parquet',index=False)
X_test.to_parquet('X_test.parquet',index=False)
y_train.to_parquet('y_train.parquet',index=False)
y_test.to_parquet('y_test.parquet',index=False)

In [188]:
def null_dict(df,grouped_cols):
    imputation_dict = {}
    # Calculate the sum of null values for each column
    null_sum = df.isnull().sum()

    # Filter columns where the sum of null values is greater than 0
    columns_with_nulls = null_sum[null_sum > 0].index

    # Select only the columns from the original DataFrame that have more than 0 null values
    null_cols = df.loc[:, columns_with_nulls].columns
    for col in null_cols:
        if (df[col].dtypes == 'int64') | (df[col].dtypes == 'float64'):
            imputation_dict.update({col : df.groupby(grouped_cols)[col].median().to_dict()})
        elif (df[col].dtypes == 'O'):
            imputation_dict.update({col : df.groupby(grouped_cols)[col].agg(pd.Series.mode).to_dict()})
                
    return imputation_dict

In [189]:
imput_dict = null_dict(X_train,group_cols)

In [190]:
def fillnulls(data,grouped_cols,imput_dict):
    imputation_dict = imput_dict
    
    col_x,col_y,col_z =grouped_cols
    
    for col in imputation_dict:
        for val in imputation_dict[col]:
            val_x,val_y,val_z = val
            data.loc[(data[col_x] == val_x) & (data[col_z] == val_z) & (data[col_z] == val_z),col] = imputation_dict[col][val]
    return data

In [191]:
X_train = fillnulls(X_train,group_cols,imput_dict)
X_test = fillnulls(X_test,group_cols,imput_dict)

In [192]:
object_cols = df.select_dtypes(include=['object']).columns

In [193]:
oneh = OneHotEncoder(sparse_output=False)
oneh.fit(X_train[object_cols])

OneHotEncoder(sparse_output=False)

In [194]:
col_names = oneh.get_feature_names_out()

In [195]:
X_train_dummies = pd.DataFrame(oneh.transform(X_train[object_cols]),columns = col_names)
X_train_dummies.index = X_train.index
X_train = pd.merge(X_train,X_train_dummies,left_index=True,right_index=True)

X_test_dummies = pd.DataFrame(oneh.transform(X_test[object_cols]),columns = col_names)
X_test_dummies.index = X_test.index
X_test = pd.merge(X_test,X_test_dummies,left_index=True,right_index=True)

In [196]:
X_train.drop(columns = object_cols,inplace = True)
X_test.drop(columns = object_cols,inplace = True)

In [197]:
def oversample_with_adasyn(X, y, target_minority_samples=50000):
    """
    Oversample the minority class using ADASYN until the minority class reaches the target_minority_samples.
    
    Parameters:
    - X: Feature matrix
    - y: Target vector
    - target_minority_samples: Desired number of samples for the minority class after oversampling
    
    Returns:
    - X_resampled, y_resampled: Feature matrix and target vector after resampling
    """
    # Identify minority class
    classes, counts = np.unique(y, return_counts=True)
    minority_class = classes[np.argmin(counts)]
    current_minority_samples = counts[np.argmin(counts)]

    # Calculate how many new samples we need to generate for the minority class
    if current_minority_samples >= target_minority_samples:
        raise ValueError("The minority class already has more or equal samples than the target_minority_samples.")
    
    n_samples_to_generate = target_minority_samples - current_minority_samples

    # Set the sampling strategy for ADASYN
    # This will tell ADASYN to generate the specified number of samples for the minority class
    sampling_strategy = {minority_class: n_samples_to_generate}

    # Create the ADASYN instance and fit it to the data
    adasyn = ADASYN(sampling_strategy=sampling_strategy, n_neighbors=5, random_state=42)
    X_resampled, y_resampled = adasyn.fit_resample(X, y)

    return X_resampled, y_resampled

In [198]:
def undersample_with_nearmiss_imbalanced(X, y, majority_samples=20000, minority_samples=10000):
    """
    Undersample the majority and minority classes separately using NearMiss.
    
    Parameters:
    - X: Feature matrix
    - y: Target vector
    - majority_samples: Desired number of samples for the majority class after undersampling
    - minority_samples: Desired number of samples for the minority class after undersampling
    
    Returns:
    - X_resampled, y_resampled: Feature matrix and target vector after resampling
    """
    # Identify majority and minority classes
    classes, counts = np.unique(y, return_counts=True)
    majority_class = classes[np.argmax(counts)]
    minority_class = classes[np.argmin(counts)]
    
    # Set the sampling strategy for NearMiss
    # This will balance the majority class with the minority class
    sampling_strategy = {majority_class: majority_samples, minority_class: minority_samples}
    nm = NearMiss(sampling_strategy=sampling_strategy)
    
    # Perform the undersampling
    X_resampled, y_resampled = nm.fit_resample(X, y)
    
    return X_resampled, y_resampled

# Perform undersampling with the specified numbers of samples
X_resampled, y_resampled = undersample_with_nearmiss_imbalanced(X_train, y_train, majority_samples=20000, minority_samples=10000)

In [199]:
X_resampled,y_resampled = oversample_with_adasyn(X_train,y_train)



In [200]:
X_train,y_train = undersample_with_nearmiss_imbalanced(X_resampled,y_resampled)

In [201]:
X_train.to_parquet('X_train_engineered.parquet',index=False)
X_test.to_parquet('X_test_engineered.parquet',index=False)
y_train.to_parquet('y_train_engineered.parquet',index=False)
y_test.to_parquet('y_test_engineered.parquet',index=False)

In [202]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [203]:
X_train_scaled = scaler.transform(X_train)
X_train = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)

In [204]:
X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)

In [205]:
# Starting Logistic Regression
logr = LogisticRegression()

#Fitting Logistic Regression
logr.fit(X_train,y_train)

#Printing errors
print('Accuracy of training is '+ str(accuracy_score(y_train,logr.predict(X_train))))
print('Precision of training is '+ str(precision_score(y_train,logr.predict(X_train))))
print('Recall of training is '+ str(recall_score(y_train,logr.predict(X_train))))
print('F1 of training is '+ str(f1_score(y_train,logr.predict(X_train))))
print('ROC AUC of training is '+ str(roc_auc_score(y_train,logr.predict(X_train))))
print()
print('Accuracy of test is '+ str(accuracy_score(y_test,logr.predict(X_test))))
print('Precision of test is '+ str(precision_score(y_test,logr.predict(X_test))))
print('Recall of test is '+ str(recall_score(y_test,logr.predict(X_test))))
print('F1 of test is '+ str(f1_score(y_test,logr.predict(X_test))))
print('ROC AUC of test is '+ str(roc_auc_score(y_test,logr.predict(X_test))))

C:\Users\37789\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of training is 0.6805
Precision of training is 0.5626320555387866
Recall of training is 0.1864
F1 of training is 0.2800270412378878
ROC AUC of training is 0.556975

Accuracy of test is 0.7595239256621629
Precision of test is 0.08531082887700535
Recall of test is 0.2099095394736842
F1 of test is 0.12131653992395437
ROC AUC of test is 0.5083164110087572


# We are trying to get this model to perform better